In [1]:
import os
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
import sys
sys.path.insert(0, '../')

import cv2
import numpy as np
from TensorRT_Inference import TRTInference

In [2]:
model = TRTInference('rtdetr_hgnetv2_x_6x_coco.trt', output_names_mapping={'tile_3.tmp_0': 'bbox_num', 'reshape2_83.tmp_0': 'bbox'}, verbose=True)
img = cv2.imread("../soccer.jpg")
org_img = img
im_shape = np.array([[float(img.shape[0]), float(img.shape[1])]]).astype('float32')
img = cv2.resize(img, (640, 640))
scale_factor = np.array([[float(640/img.shape[0]), float(640/img.shape[1])]]).astype('float32')
img = img.astype(np.float32) / 255.0
img = np.transpose(img, [2, 0, 1])  # BGR to RGB
img = np.expand_dims(img, axis=0)  # add batch dimension
inputs_dict = {
    'im_shape': im_shape,
    'image': img,
    'scale_factor': scale_factor
}
model.warmup(inputs_dict)

In [3]:
img = cv2.imread("../soccer.jpg")
org_img = img
im_shape = np.array([[float(img.shape[0]), float(img.shape[1])]]).astype('float32')
img = cv2.resize(img, (640, 640))
scale_factor = np.array([[float(640/img.shape[0]), float(640/img.shape[1])]]).astype('float32')
img = img.astype(np.float32) / 255.0
img = np.transpose(img, [2, 0, 1])  # BGR to RGB
img = np.expand_dims(img, axis=0)  # add batch dimension
inputs_dict = {
    'im_shape': im_shape,
    'image': img,
    'scale_factor': scale_factor
}
result = model(inputs_dict)['bbox']

In [4]:
confidence_threshold = 0.5
boxes = result[result[:, 1] > confidence_threshold]
for box in boxes:
    cv2.rectangle(org_img, (int(box[2]), int(box[3])), (int(box[4]), int(box[5])), (0, 255, 0), 2)
cv2.imwrite("../output/soccer_trt.jpg", org_img)

True

In [5]:
result_dicts = []
for box in boxes:
    result_dicts.append({
        'img_name': 'soccer.jpg',
        'class': 0,  # example
        'confidence': box[1],
        'ymin': box[3],
        'xmin': box[2],
        'ymax': box[5],
        'xmax': box[4]
    })